<a href="https://colab.research.google.com/github/chris-iscool/machinelearning/blob/main/Linear_Regression_Validation_and_Testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Nur für die Tensorflow Version
%tensorflow_version 2.x

In [2]:
# Wichtige imports für Tensorflow
import numpy as np
import pandas as pd
import tensorflow as tf
from matplotlib import pyplot as plt

pd.options.display.max_rows = 10
pd.options.display.float_format = "{:.1f}".format

Diesmal benutzen wir zwei Datensätze, einen Training und einen Test Satz. Im weiteren werden wir auch vom Training Satz noch einen Teil absplitten als Validation Satz.

In [3]:
housing_train_df = pd.read_csv(filepath_or_buffer="https://download.mlcc.google.com/mledu-datasets/california_housing_train.csv")
housing_test_df = pd.read_csv(filepath_or_buffer="https://download.mlcc.google.com/mledu-datasets/california_housing_test.csv")

# Der median house value wird wieder skaliert in beiden Datensätzen
scaling_factor = 1000.0
housing_train_df["median_house_value"] /= scaling_factor
housing_test_df["median_house_value"] /= scaling_factor

Wir übernehmen die build_model und train_model Methoden aus den anderen bisherigen Notebooks. Als Neuerung führen wir den Validation Split ein, womit das Modell mit einem Teil des Training Satzes trainiert und mit dem Rest validiert wird. Dies wird von den Tensorflow Methoden automatisch ausgeführt, der validation split ist also ein einfaches Argument für die Methode.

In [4]:
def build_model(eta):
  # Ein einfaches Model, ein Layer, ein Knoten (Neuron), eine Funktion
  f = tf.keras.models.Sequential()

  f.add(tf.keras.layers.Dense(units=1, input_shape=(1,)))

  # Alle weiteren Eigenschaften des 1 Neuron Model werden hier bestimmt, wie Lernen durch Root Mean Square
  f.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=eta), loss="mean_squared_error", metrics=[tf.keras.metrics.RootMeanSquaredError()])

  return f

# Training des Neurons:
def train_model(model, dataframe, feature, label, epochs_T, batchsize=None, my_validation_split=0.1):
  # Wir trainieren das Neuron für eine gegebene Anzahl an Epochen
  training_history = model.fit(x=dataframe[feature], y=dataframe[label], batch_size=batchsize, epochs=epochs_T, validation_split=my_validation_split)

  # Diesmal geben wir die history an sich zurück, nicht das trainierte weight/bias
  epochs = training_history.epoch
  states = pd.DataFrame(training_history.history)
  rms_errors = states["root_mean_squared_error"]

  return [epochs, rms_errors, training_history.history]

In [6]:
def plot_model(dataframe, feature, label, trained_weight, trained_bias):
  # Labels für die Achsen
  plt.figure()
  plt.xlabel(feature)
  plt.ylabel(label)

  # Plotte die "echten" Daten
  samples = dataframe.sample(n=200)
  plt.scatter(samples[feature], samples[label])

  # Plotte das Ergebnis der linearen Regression, also eine Gerade mit dem trainierten Gewicht und Bias der letzten Epoche
  x0 = 0
  y0 = trained_bias
  x1 = samples[feature].max()
  y1 = y0 + trained_weight * x1

  plt.plot([x0, x1], [y0, y1])
  plt.show()

def plot_loss_curves(epochs, rms_errors_training, rms_errors_validation):
  plt.figure()
  plt.xlabel("Epoch")
  plt.ylabel("Root Mean Square Error")

  # Das Element 0 wird nicht mit geplottet, da der Loss bei dem Element unnatürlich hoch ist
  plt.plot(epochs[1:], rms_errors_training[1:], label="Training loss")
  plt.plot(epochs[1:], rms_errors_validation[1:], label="Validation loss")
  plt.legend()

  rms_errors_merged = rms_errors_training[1:] + rms_errors_validation[1:]
  highest_loss = rms_errors_merged.max()
  lowest_loss = rms_errors_merged.min()
  delta = highest_loss - lowest_loss
  print(delta)

  y_axis_top = highest_loss + delta * 0.05
  y_axis_bottom = lowest_loss - delta*0.05

  plt.ylim([y_axis_bottom, y_axis_top])
  plt.show()